In [96]:
import pandas as pd

In [97]:
class Fase:
    def __init__(self, tickets, puntos, horquilla, oso, tacon, s_escudo, s_espada, s_porra, s_bombardero, s_mago):
        self.tickets = tickets
        self.puntos = puntos
        self.horquilla = horquilla
        self.oso = oso
        self.tacon = tacon
        self.s_escudo = s_escudo
        self.s_espada = s_espada
        self.s_porra = s_porra
        self.s_bombardero = s_bombardero
        self.s_mago = s_mago

# Definir los valores para 10 fases diferentes
fase1 = Fase(10, 30, 60, 20, 0, 1, 1, 0, 0, 0)
fase2 = Fase(10, 30, 0, 50, 20, 0, 0, 1, 1, 0)
fase3 = Fase(10, 30, 20, 0, 50, 0, 0, 0, 0, 1)
fase4 = Fase(12, 40, 160, 0, 0, 0, 0, 0, 0, 0)
fase5 = Fase(12, 40, 5, 4, 3, 2, 1, 0, 0, 0)
fase6 = Fase(12, 40, 6, 5, 4, 3, 2, 1, 0, 0)
fase7 = Fase(12, 100, 7, 6, 5, 4, 3, 2, 1, 0)
fase8 = Fase(15, 50, 8, 7, 6, 5, 4, 3, 2, 1)
fase9 = Fase(15, 50, 9, 8, 7, 6, 5, 4, 3, 2)
fase10 = Fase(15, 50, 10, 9, 8, 7, 6, 5, 4, 0)

fases = (fase1, fase2, fase3, fase4, fase5, fase6, fase7, fase8, fase9, fase10)

# Bonus actuales
b_punt = 1
b_hor = 1.40
b_oso = 1.15
b_taco = 1.1

for fase in fases:
    fase.puntos = int(fase.puntos * b_punt)
    fase.horquilla = int(fase.horquilla * b_hor)
    fase.oso = int(fase.oso * b_oso)
    fase.tacon = int(fase.tacon * b_taco)


In [98]:
def read_df(path = "puntos.json"):
    try:
        df = pd.read_json(path)
        return df
    except:
        print("No se ha encontrado el archivo")

In [99]:
def print_info(tickets : int, df : pd.DataFrame, fases : tuple):
    
    print(df)

    print(f"\n{tickets} disponibles\n")
    i = 1
    
    for fase in fases:
        rondas = int(tickets/fase.tickets)
        
        print(f"Fase {i} ({rondas} rondas): {fase.puntos* rondas} puntos \n")
        if fase.horquilla != 0:
            print(f"{fase.horquilla * rondas} horquillas")
        if fase.oso != 0:
            print(f"{fase.oso * rondas} osos")
        if fase.tacon != 0:
            print(f"{fase.tacon * rondas} tacones")
        if fase.s_escudo != 0:
            print(f"{fase.s_escudo * rondas} soldados escudo")
        if fase.s_espada != 0:
            print(f"{fase.s_espada * rondas} soldados espada")
        if fase.s_porra != 0:
            print(f"{fase.s_porra * rondas} soldados porra")
        if fase.s_bombardero != 0:
            print(f"{fase.s_bombardero * rondas} soldados bombardero")
        if fase.s_mago != 0:
            print(f"{fase.s_mago * rondas} soldados mago")
        print("\n")
        i += 1
        

In [100]:
def actu_valor_especifico(df:pd.DataFrame,puntos=0, horquilla=0, oso=0, tacon=0, s_espada=0, s_escudo=0, s_porra=0, s_bombardero=0, s_mago=0):
    df.Puntos_Actuales["puntos"] += puntos
    df.Puntos_Actuales["horquilla"] += horquilla
    df.Puntos_Actuales["oso"] += oso
    df.Puntos_Actuales["tacon"] += tacon
    df.Puntos_Actuales["s_escudo"] += s_escudo
    df.Puntos_Actuales["s_espada"] += s_espada
    df.Puntos_Actuales["s_porra"] += s_porra
    df.Puntos_Actuales["s_bombardero"] += s_bombardero
    df.Puntos_Actuales["s_mago"] += s_mago
    
    
    return df
    

In [101]:
def actualizar_puntos(fase, rondas, df : pd.DataFrame, path = "puntos.json"):
    bonus_pun = 1.05
    bonus_bro = 1.1
    bonus_pla = 1.2
    bonus_oro = 1
    continuar = True
    try:
        fase = int(fase)
        rondas = int(rondas)
    except:
        continuar=False
        print("La fase y/o las veces completadas no son numeros enteros")
    
    if continuar:
        if rondas > 10:
            print("No se pueden realizar tantas rondas")
            return df
        else:
            enemigos = 3 * rondas
            puntos = int(30 * bonus_pun) * rondas 
            horquilla = 0
            oso = 0
            tacon = 0
            tipo = "s_escudo"
            match fase:
                case 1:
                    horquilla = int(60 * bonus_bro) * rondas
                    oso = int(20 * bonus_pla) * rondas
                case 2:
                    oso = int(50 * bonus_pla) * rondas
                    tacon = int(20* bonus_oro) * rondas
                    tipo = "s_espada"
                case 3:
                    horquilla = int(20 * bonus_bro) * rondas
                    tacon = int(50* bonus_oro) * rondas
                    tipo = "s_porra"
                case 4:
                    enemigos = 5 * rondas
                    puntos = int(40 * bonus_pun) * rondas
                    horquilla = int(160 * bonus_bro) * rondas
                    tipo = "s_espada"
                case 5:
                    puntos = int(40 * bonus_pun) * rondas
                    oso = int(130 * bonus_pla) * rondas
                case 6:
                    puntos = int(40 * bonus_pun) * rondas
                    tacon = int(110* bonus_oro) * rondas
                    tipo = "s_espada"
                case 7:
                    puntos = int(140 * bonus_pun) * rondas
                    horquilla = int(20 * bonus_bro) * rondas
                    oso = int(20 * bonus_pla) * rondas
                    tacon = int(20* bonus_oro) * rondas
                    tipo = "s_porra"
                case 8:
                    puntos = int(45 * bonus_pun) * rondas
                    horquilla = int(300 * bonus_bro) * rondas
                case 9:
                    puntos = int(45 * bonus_pun) * rondas
                    oso = int(240 * bonus_pla) * rondas
                    tipo = "s_espada"
                case 10:
                    puntos = int(45 * bonus_pun) * rondas
                    tacon = int(200* bonus_oro) * rondas
                    tipo = "s_porra"
                case other:
                    print("Esa fase no existe")
                    continuar = False
            
            if continuar:          
                if tipo == "s_porra":
                    df = actu_valor_especifico(df, puntos, horquilla, oso, tacon, s_porra=enemigos)
                elif tipo == "s_espada":
                    df = actu_valor_especifico(df, puntos, horquilla, oso, tacon, s_espada=enemigos)
                elif tipo == "s_escudo":
                    df = actu_valor_especifico(df, puntos, horquilla, oso, tacon, s_escudo=enemigos)
                elif tipo == "s_bombardero":
                    df = actu_valor_especifico(df, puntos, horquilla, oso, tacon, s_bombardero=enemigos)
                else:
                    df = actu_valor_especifico(df, puntos, horquilla, oso, tacon, s_mago=enemigos)
                    
                df["Puntos_Restantes"] = df.Puntos_Maximos - df.Puntos_Actuales
                df.to_json(path)  
            
    return df
                    

In [102]:
df = read_df()
decision = input("Que quieres hacer: 1 = comprobar puntos, 2 = aplicar puntos, otros = salir")
try: 
    decision = int(decision)
except:
    pass

match decision:
    case 1:
        tickets = input("Tickets disponibles")
        try:  
            tickets = int(tickets)
            if 180 >= tickets > 0:
                print_info(tickets, read_df(), fases)
            else:
                print("No es una cantidad permitida")
        except ValueError: 
            print("No son numeros enteros")
                 
    case 2: 
        fase, veces = input("Que fase has completado y cuantas rondas, separado por un espacio").split(" ")
        actualizar_puntos(fase, veces, read_df(), "puntos.json")
    
    case other:
        print("Adios")

              Puntos_Actuales  Puntos_Maximos  Puntos_Restantes
puntos                      0           10000             10000
horquilla                   0           10000             10000
oso                         0           10000             10000
tacon                       0           10000             10000
s_escudo                    0              50                50
s_espada                    0              50                50
s_porra                     0              50                50
s_bombardero                0              50                50
s_mago                      0              50                50

10 disponibles

Fase 1 (1 rondas): 30 puntos 

84 horquillas
23 osos
1 soldados escudo
1 soldados espada


Fase 1 (1 rondas): 30 puntos 

57 osos
22 tacones
1 soldados porra
1 soldados bombardero


Fase 1 (1 rondas): 30 puntos 

28 horquillas
55 tacones
1 soldados mago


Fase 1 (1 rondas): 400 puntos 

5 horquillas
3 osos
2 tacones
1 soldados escudo


Fase 